In [1]:
from FractalTree import *
from parameters import Parameters

param=Parameters()
param.save = False
param.meshfile = "data/sphere.obj"

branches, nodes = Fractal_Tree_3D(param)

no nodes to consider
no nodes to consider
no nodes to consider


In [28]:
import matplotlib.pyplot as plt

In [86]:
# end nodes
nn,cnt = np.unique(np.concatenate([ np.array(b.nodes)[[0,-1]] for b in branches.values() if len(b.nodes) > 1]),return_counts=True)
enodes = nn[cnt==1]

In [117]:
# node to edge map
pts = np.array(nodes.nodes)

cv = 1.0 # [m/s]

act = np.empty(pts.shape[0])
act.fill(np.inf)
act[enodes[0]]  = 0.0
#act[enodes[-1]] = 0.0

# we assume no activation in the middle of the branch
for k in range(30):
    act_old = act.copy()
    # iterate over the branches
    # NOTE this could be done in parallel
    for branch in branches.values():
        # points in the branch
        bn = branch.nodes
        bp = pts[bn,:]
        # length of each segment
        le = np.linalg.norm(np.diff(bp,axis=0),axis=1)
        # activation from first node
        dl = np.r_[0.0,np.cumsum(le)] / cv
        # update all nodes in the branch from left and right
        act[bn] = np.minimum( dl + act[bn[0]], dl[::-1] + act[bn[-1]] )

    err = np.linalg.norm(act-act_old)
    print(k,np.linalg.norm(act-act_old))
    if err < 1e-8: break
#plt.plot(act[bn])


0 inf
1 0.0


In [118]:
import meshio

edges = np.array(sum((list(zip(b.nodes[0:-1],b.nodes[1:])) for b in branches.values() if len(b.nodes) > 1),[]))
enod  = np.zeros_like(act,dtype=int)
enod[enodes] = 1

mesh = meshio.Mesh(points=pts,cells={'line':edges},point_data={'activation':act,'endnode':enod})

#mesh = meshio.read("sphere-line.vtu")
#mesh.point_data['distance'] = dd*cv
mesh.write("sphere-line-act.vtu")


In [88]:
from fimpy.solver import FIMPY

Import of Cupy failed. The GPU version of fimpy will be unavailble. Message: No module named 'cupy'


In [178]:
sphere = meshio.read("data/sphere.obj")

spts,selm = sphere.points,sphere.cells[0].data
velocity_e = np.ones(selm.shape[0])
D = 0.1*np.eye(3)[np.newaxis] * velocity_e[..., np.newaxis, np.newaxis]

fim = FIMPY.create_fim_solver(spts,selm,D)
phi = fim.comp_fim([0], [0.0])

sphere.point_data['activation'] = phi
sphere.write("sphere-act.vtu")

In [179]:
# entry points
epts = pts[enodes[1:],:]
eact = act[enodes[1:]]

from scipy.spatial import cKDTree as kdtree
t = kdtree(sphere.points)

# closest points in the mesh
d,i = t.query(epts)
# some points may overlap, take minimum
iu,idx,inv,c = np.unique(i,return_inverse=True,return_index=True,return_counts=True)
eactr = eact[idx]
for ii in range(eact.shape[0]):
    eactr[inv[ii]] = min(eact[ii],eactr[inv[ii]])

phi = fim.comp_fim(iu, eactr)
sphere.point_data['activation'] = phi
sphere.write("sphere-act.vtu")

# TBD: TK version with activation within element
# check for element in mesh
#from Mesh import Mesh
#m = Mesh("data/sphere.obj",useVTK=True)
#epts[0,:], m.project_new_point( epts[0,:] )

#print(epts,eact)

In [194]:
import vtk
from vtkmodules.numpy_interface import dataset_adapter as dsa

reader = vtk.vtkXMLUnstructuredGridReader()
reader.SetFileName("sphere-act.vtu")
reader.Update()

reader2 = vtk.vtkXMLUnstructuredGridReader()
reader2.SetFileName("sphere-line-act.vtu")
reader2.Update()

probe = vtk.vtkProbeFilter()
probe.SetSourceData(reader.GetOutput())
probe.SetInputData(reader2.GetOutput())
probe.Update()

da = dsa.WrapDataObject(probe.GetOutput())
vv = da.PointData['activation']


VTKArray([2.0117457, 1.9839078, 1.95607  , ..., 1.5062255, 1.5122421,
          1.5188162], dtype=float32)

In [203]:
loc = vtk.vtkCellLocator()
loc.SetDataSet(reader.GetOutput())
loc.BuildLocator()

In [208]:
cellId = vtk.reference(0)
c = [0.0, 0.0, 0.0]
subId = vtk.reference(0)
d = vtk.reference(0.0)
loc.FindClosestPoint(epts[0,:], c, cellId, subId, d)

genCell = vtk.vtkGenericCell()
pcoords = [0.0,0.0,0.0]
weights = [0,0,0,0,0]
loc.FindCell(epts[0,:],1e-4,genCell,pcoords,weights)

#epts[0,:],c,cellId

print(pcoords,weights)

[0.20003964128607474, 0.17934345062728715, 0.0] [0.6206169080866382, 0.20003964128607474, 0.17934345062728715, 0.0, 0.0]


In [212]:
# vtk data for the junctions
PMJPoints = vtk.vtkPoints()
for p in epts:
    PMJPoints.InsertNextPoint(p)
PMJunc = vtk.vtkPolyData()
PMJunc.SetPoints(PMJPoints)

probe = vtk.vtkProbeFilter()
probe.SetSourceData(reader.GetOutput())
probe.SetInputData(reader2.GetOutput())
probe.Update()

da = dsa.WrapDataObject(probe.GetOutput())
vv = da.PointData['activation']

print(PMJunc)


vtkPolyData (0x7f7d80c20880)
  Debug: Off
  Modified Time: 474666
  Reference Count: 1
  Registered Events: (none)
  Information: 0x600002365cc0
  Data Released: False
  Global Release Data: Off
  UpdateTime: 0
  Field Data:
    Debug: Off
    Modified Time: 474635
    Reference Count: 1
    Registered Events: (none)
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
  Number Of Points: 330
  Number Of Cells: 0
  Cell Data:
    Debug: Off
    Modified Time: 474638
    Reference Count: 1
    Registered Events: 
      Registered Observers:
        vtkObserver (0x6000038657d0)
          Event: 33
          EventName: ModifiedEvent
          Command: 0x60000152c690
          Priority: 0
          Tag: 1
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
    Copy Tuple Flags: ( 1 1 1 1 1 0 1 1 1 1 1 )
    Interpolate Flags: ( 1 1 1 1 1 0 0 1 1 1 1 )
    Pass Through Flags: ( 1 1 1 1 1 1 1 1 1 1 1 )
    Scalars: (none)
    Vectors: (none)
   